In [1]:
%cd /afs/crc.nd.edu/user/m/msaebi/Public/chemistry/yield_rxn

/afs/crc.nd.edu/user/m/msaebi/Public/chemistry/yield_rxn


In [2]:
import argparse
import logging
import os
import json
import math
import warnings
import pickle

import pandas as pd
import numpy as np
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit

import rdkit
from rdkit.ML.Descriptors import MoleculeDescriptors
import rdkit.Chem as Chem
from rdkit.Chem import Descriptors

warnings.filterwarnings("ignore")

In [3]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def clean(num):
    if num is None:
        return 0
    if num=='[]':
        return ''
    elif math.isnan(float(num)):
        return 0.0
    else:
        return round(float(num),5)
    
def get_func(mol_json,key):
        if mol_json.get(key,0) is None:
            return -1
        else:
            return round(float(mol_json.get(key,0)),5)
        
def build_rdkit_features(smile,comp_name):
    descriptor_list=[x[0] for x in Descriptors._descList]
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_list)
    rdkit_feature_names=[comp_name+'_'+j for j in calculator.GetDescriptorNames()]
    
    
    mol=Chem.MolFromSmiles(smile)
    mol= Chem.AddHs(mol)
    rdkit_feature_values= calculator.CalcDescriptors(mol)
    rdkit_feats = dict(zip(rdkit_feature_names,rdkit_feature_values))       
    
    return rdkit_feats

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("-dn", "--dataset_name", type=str, default="dy",required=True, help="dataset name. Options: az (AstraZeneca),dy (Doyle),su (Suzuki)")
parser.add_argument("-dp","--dataset_path", type=str, default='./data/', help="dataset name")
parser.add_argument("-rdkit", "--use_rdkit_feats", required=True, type=int, help="Use rdkit discriptors or not")
parser.add_argument("-tr", "--test_ratio", type=float, required=True, default=0.3, help="test ratio for split")
parser.add_argument("-rs", "--random_state",  type=float, default=0, help="Random state for RF model")

args = parser.parse_args(args=["-dn",'az',
                        "-rdkit","0",
                        '-tr',"0.4"])

In [5]:
error_reaction_ids=set();
data_type=args.dataset_name
input_data= data_type+'_reactions_data.json'
use_rdkit_features= args.use_rdkit_feats
input_data_path = os.path.join(args.dataset_path,data_type,'raw',input_data)

ext = '_no_rdkit' if not use_rdkit_features else '_rdkit'
output_fn= ''.join([data_type,ext ,'.csv'])
output_path= os.path.join(args.dataset_path,data_type,'processed')
if not os.path.exists(output_path):
    os.mkdir(output_path)
output_file= os.path.join(output_path,output_fn)
train_test_idx_file= os.path.join(output_path,'train_test_idxs.pickle')

print("\n\nReading data from: ",input_data_path)
print("Using rdkit features!") if use_rdkit_features else print("Not using rdkit features!")


data_dict=defaultdict(lambda: defaultdict(float))



Reading data from:  ./data/az/raw/az_reactions_data.json
Not using rdkit features!


In [6]:
#get rdkit descriptors
descriptor_list=[x[0] for x in Descriptors._descList]
                
with open(input_data_path) as datafile:
    lines = json.load(datafile)

    for line in lines:
        solvent_key='solvent' if 'solvent' in line.keys() else 'Solvent' #case consitencies in data
        base_key= 'Base' if 'Base' in line.keys() else 'base' #case consitencies in data
        
        
        if data_type=='az':
            name= reaction_num = line["reaction_Num"]
            r_yield=line['yield']['yield']
        
        elif data_type in ['dy','su']:
            name= reaction_num = line["Id"]
            r_yield=line['yield']
        
        data_dict[name]["id"] = name
        reactants=line['reactants']  
        reactant_smiles=[]
        for comp in reactants:
            smiles= comp.get('smiles','')
            reactant_smiles.append(smiles)
            
        product_smiles = line.get("product",{}).get('smiles','')
        base_smile = line.get(base_key,{}).get('smiles','')
        
        
        # for Doyle data, solvent is the same for all reactions
        solvent_smile = 'CS(=O)C' if data_type=='dy' else line.get(solvent_key,[''])[0]
        if solvent_smile=='' or  solvent_smile==0:
            print(name)
            continue
            
        data_dict[name]["reactant_smiles"] = '.'.join(reactant_smiles)
        data_dict[name]["solvent_smiles"] = solvent_smile
        data_dict[name]["base_smiles"] = base_smile
        data_dict[name]["product_smiles"] = product_smiles


        ########### other general reaction features 
        # temperature is different only in AZ data
        if data_type=='az': data_dict[name]['temperature'] = clean(line.get('temperature',0))
        elif data_type=='dy': data_dict[name]['temperature']=60.0
        elif data_type=='su': data_dict[name]['temperature']=100.0
            
        data_dict[name]['base_pka'] = clean(line.get(base_key,{}).get('Pka of Base-H',0))
        data_dict[name]['base_atom_cat'] = clean(line.get(base_key,{}).get('Atomic_number_Cation',0))   
        
        ########## other features in AZ data
        data_dict[name]['reaction_scale'] = clean(line.get('scale',0))
        data_dict[name]['base_amount'] = clean(line.get('base_amount',0))
        data_dict[name]['catalyst_amount'] = clean(line.get('catalyst_amount',0))
        data_dict[name]['reaction_volume'] = clean(line.get('volume',0))
        ##########
        
        #get solvent values
        solvent_vec= line.get(solvent_key,[])
        
        if solvent_vec !=[]:
            for i in range(1,len(solvent_vec)):
                data_dict[name]['solvent_'+str(i)]=float(solvent_vec[i])
        
        for mol_idx in range(len(reactants)):
            mol = reactants[mol_idx]     
            category = mol.get('category','')
            mol_smiles = mol.get('smiles','')
            #doing this helps with mapping reactions form Su to Dy and Az together
            #if category=='Boronic Acid': category='Amine' 
            
            #calculate rdkit features for reactants and integrate into data_dict
            if use_rdkit_features:
                rdkit_feats_curr_mol = build_rdkit_features(mol_smiles,category)
                for key, value in rdkit_feats_curr_mol.items():
                    data_dict[name][key] = value
            
            
            vib_modes = mol.get('vib_modes',[[0.0,0.0],[0.0,0.0],[0.0,0.0]])
            atoms =mol.get('atoms',{})
            
            data_dict[name][category] = mol.get('name','')
            data_dict[name][category +'_molecular_weight'] = get_func(mol,'molecular_weight')
            data_dict[name][category +'_molecular_volume'] = get_func(mol,'volume')
            data_dict[name][category +'_surface_area'] = get_func(mol,'surface_area')
            
            data_dict[name][category +'_ovality'] = get_func(mol,'ovality')
            data_dict[name][category +'_hardness'] = get_func(mol,'hardness')
            data_dict[name][category +'_dipole_moment'] = get_func(mol,'dipole_moment')
            data_dict[name][category +'_electronegativity'] = get_func(mol,'electronegativity')
            data_dict[name][category +'_E_HOMO'] = get_func(mol,'HOMO_energy')
            data_dict[name][category +'_E_LOMO'] = get_func(mol,'LUMO_energy')
            
            for n in range(len(vib_modes)):
                data_dict[name][category + '_V'+str(n)+'_frequency'] = round(float(vib_modes[n][0]),5)
                data_dict[name][category + '_V'+str(n)+'_intensity'] = round(float(vib_modes[n][1]),5)
            
            for atom in atoms:
                if 'H' not in atom['name']: #exculding hydrogen for now
                    if 'partial_charge' in atom:
                        data_dict[name][category+'_.'+atom['name']+'_electrostatic_charge']=clean(atom['partial_charge'])
                                     
                    if 'nmr_shift' in atom:
                        data_dict[name][category+'_.'+atom['name']+'_NMR_shift']= clean(atom['nmr_shift'])
                                                
            ######################### 
            #get rdkit fetures for base, solvent, product
            if use_rdkit_features:
                all_comps = [solvent_smile,base_smile,product_smiles]
                comp_map= {0:'solvent',1:'base',2:'product'}
                rdkit_feats_combined={}
                for i,smile in enumerate(all_comps):
                    if smile not in ['',' ',0]:
                        rdkit_feats_combined = build_rdkit_features(smile,comp_map[i])            

                        for key, value in rdkit_feats_combined.items():
                            data_dict[name][key] = value
                
            ###############################            
        if isfloat(r_yield):                
            data_dict[name]['yield'] = round(float(r_yield),5)      
        else:
            del data_dict[name]
            error_reaction_ids.add(name)
            
            
print(f"\nNumber of reactions with problematic yield: {len(error_reaction_ids)}")
print(f"Number of valid reactions in data dict: {len(data_dict)}")




Number of reactions with problematic yield: 0
Number of valid reactions in data dict: 750


In [27]:
#Convert data_dict to pandas dataframe
df_o=pd.DataFrame.from_dict(data_dict, orient='index')
df_o=df_o.reset_index()

smiles_features = ["reactant_smiles","solvent_smiles","base_smiles","product_smiles"]
categorical =list( df_o.columns[ ~( (df_o.dtypes.values == np.dtype('float64')) | (df_o.dtypes.values == np.dtype('int64')))])
categorical = [i for i in categorical if i not in smiles_features]

zero_val_features = list(df_o.columns[(df_o == 0).all()])

print(f"Number of all features: {df_o.shape[1]}")
print(f"Number of catgorical features minus smiles features: {len(categorical)-4}")
print(f"Number of zero-value features: {len(zero_val_features)}")


to_drop=zero_val_features+categorical+['index']
print(f"\nDropping {len(to_drop)} zero-val and categorical features...")

df= df_o.copy()
df.drop(to_drop, axis=1, inplace=True)
feature_cols= [f for f in df.columns if f not in smiles_features+['yield']]
new_columns= ['yield']+ smiles_features + feature_cols
assert len(new_cols)== df_o.shape[1]-len(to_drop)
print(f"Number of features after dropping: {len(new_cols)}")


df.fillna(0, inplace=True)
df=df[new_columns]

print(f"\nWriting csv file to: {output_file}")
df.to_csv(output_file)



#creating 10 random data splits -shuffled
print(f"\nWriting the train/test split indexs\n")
rs = ShuffleSplit(n_splits=10, test_size=args.test_ratio, random_state= args.random_state)
idx_dict= {'train_idx':{},'test_idx':{}}
i=1
for train_index, test_index in rs.split(df):
    if i==1:
        print(f"Num train idxs: {len(train_index)}, Num test idxs: {len(test_index)}")
    idx_dict['train_idx'][i] = train_index
    idx_dict['test_idx'][i]= test_index
    i+=1
    
    
with open(train_test_idx_file, 'wb') as handle:
    pickle.dump(idx_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Number of all features: 392
Number of catgorical features minus smiles features: 4
Number of zero-value features: 17

Dropping 26 zero-val and categorical features...
Number of features after dropping: 366

Writing csv file to: ./data/az/processed/az_no_rdkit.csv

Writing the train/test split indexs

Num train idxs: 450, Num test idxs: 300


In [54]:
df.shape

(750, 366)

In [85]:
selected_features_fn = os.path.join(args.dataset_path,data_type,'rf_results','selected_feats.txt')
selected_features = open(selected_features_fn,'r').readlines()[0].split(',')
input_split_idx_file = os.path.join(output_path,'train_test_idxs.pickle')

with open(input_split_idx_file, 'rb') as handle:
    idx_dict = pickle.load(handle)





In [90]:
train_set= df.iloc[idx_dict['train_idx'][1]]
test_set = df.iloc[idx_dict['test_idx'][1]]
selected_features = open(selected_features_fn,'r').readlines()[0].split(',')



smiles_feature_names = ["id","yield","reactant_smiles","solvent_smiles","base_smiles","product_smiles"]
domain_feature_names = [f for f in df.columns if f not in smiles_feature_names]
if True:
    domain_feature_names = [f for f in domain_feature_names if f in selected_features]

    
train_set_domain = train_set[domain_feature_names]
train_set_smile = train_set[smiles_feature_names]

test_set_domain = test_set[domain_feature_names]
train_set_smiles = test_set[smiles_feature_names]


scaler = StandardScaler()

train_set_domain_scaled = pd.DataFrame(scaler.fit_transform(train_set_domain),columns = domain_feature_names)
test_set_domain_scaled = pd.DataFrame(scaler.transform(test_set_domain),columns = domain_feature_names)





In [86]:
test_set = df.iloc[idx_dict['test_idx'][1]]


df_smiles= df[smiles_feature_names]
df_domain= df[domain_features_selected]

scaler = StandardScaler()
train_set_scaled = pd.DataFrame(scaler.fit_transform(df_domain),columns = domain_feature_names)



,temperature,reaction_scale,base_amount,reaction_volume,solvent_1,solvent_2,amine_molecular_weight,amine_molecular_volume,amine_surface_area,amine_ovality,...,halide_.B1_electrostatic_charge,halide_.B1_NMR_shift,amine_.I1_electrostatic_charge,amine_.I1_NMR_shift,amine_.Br1_electrostatic_charge,amine_.Br1_NMR_shift,amine_.F4_electrostatic_charge,amine_.F4_NMR_shift,ligand_.O3_electrostatic_charge,ligand_.O3_NMR_shift
0,-0.297958,-0.345937,-0.322621,-0.295176,-0.401737,-1.070413,0.785566,0.392854,0.600859,0.810530,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
1,0.109089,-0.335160,-0.310113,-0.275890,-0.401737,-1.070413,0.425090,0.710341,0.554753,0.322485,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
2,2.144324,-0.294744,-0.239378,-0.266248,-0.413753,0.266105,-0.069352,-0.313404,-0.232503,0.013239,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
3,-1.112053,0.164921,0.248000,-0.015533,-0.401737,-1.070413,0.366724,0.360388,0.248476,0.139985,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
4,1.737277,-0.332465,-0.280352,-0.285533,-0.401737,-1.070413,0.569917,0.303613,0.467029,0.661143,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,-0.705006,-0.301749,-0.243260,-0.285533,-0.413753,0.266105,2.671656,2.318315,2.746411,2.172779,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
446,-0.297958,1.343452,-0.190209,0.948756,-0.413753,0.266105,0.076055,0.039715,0.029947,0.097877,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
447,0.516136,-0.200440,0.029758,-0.208390,0.078930,0.940494,0.366724,0.360388,0.248476,0.139985,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0
448,0.312612,1.823056,-0.252749,0.418397,-0.401737,-1.070413,0.569917,0.829183,0.467026,0.029214,...,-0.047193,0.0,0.051846,0.0,0.094703,0.0,0.047193,0.047193,0.047193,0.0


In [59]:
df=pd.read_csv(output_file,index_col=0)
smiles_feature_names = ["id","yield","reactant_smiles","solvent_smiles","base_smiles","product_smiles"]
domain_feature_names = [f for f in df.columns if f not in smiles_features]
domain_features_selected = [f for f in domain_feature_names if f in selected_features]
        
df_smiles= df[smiles_feature_names]
df_domain= df[domain_features_selected]

(750, 213)

In [63]:
print(len(domain_feature_names),len(domain_features_selected ))

360 213


In [72]:
with_domain=use_domain if 'rdkit' in use_domain else 'no_domain'
print(with_domain)

no_domain


In [49]:
df=pd.read_csv(output_file,index_col=0)
smiles_features = ["id","yield","reactant_smiles","solvent_smiles","base_smiles","product_smiles"]
domain_features = [f for f in df.columns if f not in smiles_features]
df_smiles= df[smiles_features]
df_domain= df[domain_features]

In [53]:
for i in range(3):
    reactant_smiles= df_smiles.iloc[i]["reactant_smiles"]
    solvent_smiles = df_smiles.iloc[i]["solvent_smiles"]
    base_smiles =  df_smiles.iloc[i]["base_smiles"]
    product_smiles =  df_smiles.iloc[i]["product_smiles"]
    rxn_yield= df_smiles.iloc[i]["yield"]
    domain_features = df_domain.iloc[i].values
    


CC(C)N1CCNCC1.CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=C(C=C(C=C3)F)F)C(=O)OCC)Br.C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=C(C4=CC=CC=C4C=C3)C5=C(C=CC6=CC=CC=C65)P(C7=CC=CC=C7)C8=CC=CC=C8 C1COCCO1 65.39
[ 1.1000000e+02  8.9000000e-04  2.2200000e-03  0.0000000e+00
  2.2500000e+00  7.3000000e-01  1.2822000e+02  2.0498672e+02
  3.1460043e+02  4.0669000e-01  2.8236000e-01  4.4460000e-01
  5.9200000e-02 -3.4155000e-01  2.2316000e-01 -6.4833000e-01
  1.8995430e+02 -9.4084000e-01  1.6611000e+02  1.9140000e-02
  3.7934500e+01  1.9140000e-02  3.7934600e+01  5.0544000e-01
  4.2939600e+01  3.3155000e-01  3.4327500e+01  3.3157000e-01
  3.4327400e+01 -3.1488000e-01  1.0407800e+01 -3.1486000e-01
  1.0408200e+01  4.5130000e+02  4.6034872e+02  9.7115454e+02
  7.3206000e-01  1.7835000e-01  2.7437000e+00  1.2736000e-01
 -3.0570000e-01  5.0990000e-02  1.7417290e+03  2.1849540e+02
  7.5415070e+02  1.9964000e+00  7.9919360e+02  1.8322900e+01
 -7.1110000e-02  0.0000000e+00 -2.9685000e-01 -1.2386300e+02
 -3.1135000e-01 -1.1681450

In [31]:
#df=pd.read_csv(output_file,index_col=0)
import csv
reactions={}
with open(output_file) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        reactions.append(row)
print("Number of all moves: ",len(reactions))

Number of all moves:  750


In [32]:
reactions[0]

OrderedDict([('', '0'),
             ('yield', '65.39'),
             ('reactant_smiles',
              'CC(C)N1CCNCC1.CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=C(C=C(C=C3)F)F)C(=O)OCC)Br.C1=CC=C(C=C1)P(C2=CC=CC=C2)C3=C(C4=CC=CC=C4C=C3)C5=C(C=CC6=CC=CC=C65)P(C7=CC=CC=C7)C8=CC=CC=C8'),
             ('solvent_smiles', 'C1COCCO1'),
             ('base_smiles', 'C(=O)([O-])[O-].[Cs+].[Cs+]'),
             ('product_smiles',
              'CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=C(C=C(C=C3)F)F)C(=O)OCC)N4CCN(CC4)C(C)C'),
             ('id', '1'),
             ('temperature', '110.0'),
             ('reaction_scale', '0.00089'),
             ('base_amount', '0.00222'),
             ('reaction_volume', '0.0'),
             ('solvent_1', '2.25'),
             ('solvent_2', '0.73'),
             ('amine_molecular_weight', '128.22'),
             ('amine_molecular_volume', '204.98672'),
             ('amine_surface_area', '314.60043'),
             ('amine_ovality', '0.40669'),
             ('amine_hardness', '0.2823